# Matching algorithm

### Prerequisites

Download processed datasets via `git lfs fetch --include "data/processed/*.csv"`

or create them from scratch with [preprocess.ipynb](./preprocess.ipynb)

In [39]:
!pip install pandas
!pip install scikit-learn
!pip install nltk
!pip install sentence-transformers
!pip install numpy

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [40]:
import os
import os.path as path
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import numpy as np

In [41]:
root_dir = path.dirname(path.abspath(os.getcwd()))
data_dir = path.join(root_dir, 'data')
processed_data_dir = path.join(data_dir, 'processed')

processed_vacancies_path = path.join(processed_data_dir, 'vacancies.csv')
processed_cvs_path = path.join(processed_data_dir, 'cvs.csv')

processed_cvs_path

'/Users/alv.popov/prj/DL-CASE2/data/processed/cvs.csv'

In [42]:
processed_vacancies = pd.read_csv(processed_vacancies_path)
processed_vacancies

,text
0,Project R’n’D / Технолог R’n’D . MF Kitchen — ...
1,"Продавец-кассир (Москва, Богданова, 16) . Обяз..."
2,"Продавец-кассир (Москва, Новогиреевская, 11/36..."
3,"Product owner бизнес–стрима ""Аналитика и Отчет..."
4,Business Development Director (CTO) . Студия О...
...,...
96220,"Оператор 1C . ТД «ЛФБ» - семейная компания, ..."
96221,Стажер Управления финансирования торговли . КР...
96222,Преподаватель математики/информатики . Обязанн...
96223,Программист С++ / Linux . Требования: Знание ...


In [43]:
processed_cvs = pd.read_csv(processed_cvs_path)
processed_cvs

,text
0,"Мужчина , 39 лет , родился 27 ноября 1979 . С..."
1,"Мужчина , 60 лет , родился 20 марта 1959 . Те..."
2,"Женщина , 36 лет , родилась 12 августа 1982 ...."
3,"Мужчина , 38 лет , родился 25 июня 1980 . Веб..."
4,"Женщина , 26 лет , родилась 3 марта 1993 . Ре..."
...,...
44739,"Мужчина , 30 лет , родился 17 января 1989 . Ф..."
44740,"Мужчина , 27 лет , родился 5 марта 1992 . Сис..."
44741,"Женщина , 48 лет , родилась 26 декабря 1970 ...."
44742,"Мужчина , 24 года , родился 6 октября 1994 . ..."


In [44]:
model = SentenceTransformer('cointegrated/LaBSE-en-ru')

In [45]:
vacancy_embeddings = model.encode(processed_vacancies['text'][0])
cv_embeddings = model.encode(processed_cvs['text'][0])

cosine_similarity([vacancy_embeddings], [cv_embeddings])[0][0]

0.39085472

In [46]:
n_items = 10

sampled_vacancies = processed_vacancies.sample(n_items)
sampled_cvs = processed_cvs.sample(n_items)

vacancy_embeddings = model.encode(sampled_vacancies['text'].tolist())
cv_embeddings = model.encode(sampled_cvs['text'].tolist())

results = cosine_similarity(vacancy_embeddings, cv_embeddings)
print('\n'.join(map(lambda x: '\t'.join(map(str, x)), results[:10, :10])))

0.5560857	0.60102564	0.46651688	0.5941452	0.5036321	0.6468186	0.5476116	0.459289	0.44116914	0.54149556
0.61388296	0.70549953	0.53421474	0.6247531	0.56836885	0.7068291	0.51328135	0.6067818	0.5182052	0.60286605
0.5653487	0.5213471	0.5465622	0.5056963	0.38834623	0.45620775	0.40628546	0.47590825	0.43251607	0.3651454
0.5163249	0.586914	0.49920782	0.48795044	0.4547744	0.59781855	0.50101376	0.5595152	0.42453918	0.5277907
0.5464641	0.66892964	0.4538477	0.62875795	0.5230783	0.6591692	0.54398483	0.47688416	0.396113	0.6026411
0.56431985	0.55319715	0.526634	0.5501336	0.46666148	0.5936499	0.51517695	0.49278396	0.45049667	0.47882527
0.5442399	0.56348866	0.51948136	0.5150732	0.44240257	0.5680465	0.55506015	0.505646	0.41980726	0.46269614
0.59228134	0.6321138	0.53064775	0.55034703	0.5571159	0.65554684	0.46615437	0.5831052	0.47999945	0.60515904
0.51233673	0.46565437	0.46707112	0.45673144	0.5041768	0.53939414	0.46730494	0.41021803	0.3774895	0.52403057
0.5447037	0.57954174	0.4786473	0.5424749	0.52972597	0

In [47]:
ind = np.where(results > 0.6)
ind

(array([0, 0, 1, 1, 1, 1, 1, 1, 4, 4, 4, 4, 7, 7, 7]),
 array([1, 5, 0, 1, 3, 5, 7, 9, 1, 3, 5, 9, 1, 5, 9]))

In [49]:
text1 = sampled_vacancies['text'].iloc[0]
text2 = sampled_cvs['text'].iloc[1]

embs1 = model.encode(text1)
embs2 = model.encode(text2)

metric = cosine_similarity([embs1], [embs2])[0][0]
display(text1, text2, metric)

'Менеджер по оптовым продажам . Обязанности: Поиск и привлечение новых клиентов ( Холодные звонки + клиентская база). Продажа сыпучих материалов ( песок , щебень, вывоз грунта и т.д.). Работа с базой клиентов компании и развитие ее. Обработка входящих звонков. Подготовка, заполнение шаблонов договора поставки и перевозки. Контроль дебиторской задолженности Оперативное решение всех возникших вопрос с клиентом, контроль своевременности поставки Сопровождение клиентов Ведение первичной документации по клиенту Выезд на объект на прямые продажи.   Требования: Образование: среднее специальное, высшее.  Опыт работы в продажах нерудных материалах приветствуется. Наличие собственной клиентской базы приветствуется и рассматривается как серьезное преимущество. Уверенное пользование : ПК, MS Office. 1C и орг техникой. Опыт ведение переговоров, на уровне первых лиц Стрессоустойчивость , исполнительность. грамотная речь, желание расти в компании так и саморазвиваться Условия: Возможна дистанционная 

'Мужчина ,  29 лет , родился 16 ноября 1989 . Директор ИТ  / Начальник отдела ИТ / Технический директор / CTO / CIO . Москва ,  м. Речной вокзал , не готов к переезду , готов к командировкам . полная занятость . полный день . Опыт работы 11 лет 8 месяцев  Директор ИТ / Начальник отдела ИТ / Технический директор / CTO / CIO 250 000 руб. Информационные технологии, интернет, телеком CTO, CIO, Директор по IT Занятость: полная занятость График работы: полный день Опыт работы 11 лет 8 месяцев Июль 2018 — по настоящее время 10 месяцев Мобильные ТелеСистемы (МТС) Москва , www.mts.ru Телекоммуникации, связь ... Мобильная связь Директор центра ИТ Задачи: - Руководство подразделением из 100 человек; - Организация работы по предоставлению всего спектра услуг в сфере ИТ для внутренних и внешних заказчиков (одновременное сопровождение ~4000000 внешних клиентов и 10000 внутренних пользователей); -Сопровождение инфраструктурных и клиентских инцидентов, а так же контроль проведения плановых работ на пр

0.6010257